## Local Function Calling
The **LocalCodeTool** ant demonstrates the use of .Net functions as tools! You don't have to use a Web API. 

## First steps
- First, download AntRunLib from Nuget
- At least once, setup the enviroment using **[0-AI-settings](0-AI-settings.ipynb)**

In [6]:
#r "nuget: AntRunnerLib, 0.6.5"

using AntRunnerLib;
using AntRunnerLib.Identity;
using static AntRunnerLib.ClientUtility;
using System.IO;

#!import config/Settings.cs 

var envVariables = Settings.GetEnvironmentVariables();
foreach (var kvp in envVariables)
{
    Environment.SetEnvironmentVariable(kvp.Key, kvp.Value);
}

var config = AzureOpenAIConfigFactory.Get();
var client = GetOpenAIClient(config);


Installed Packages AntRunnerLib, 0.6.5

## Load the assembly containing the functions
AntLoader.LoadAssembly ensures the assembly is loaded and findable by AntRunnerLib

**Don't load any assemblies you don't fully trust!**

In [7]:
AntLoader.LoadAssembly(@".\AssistantDefinitions\LocalCodeTool\TestAssistant.dll");

## Ensure the LocalCodeTool assistant exists
The definition of this assistant is located in the ".\AssistantDefinitions\LocalCodeTool" folder

".\AssistantDefinitions\" is a default path. You can override it by setting the **ASSISTANTS_BASE_FOLDER_PATH** enviroment variable.

"LocalCodeTool" contains the following files:
```
│   instructions.md - The instructions
│   LocalFunction.cs - The code that defines the two functions for your reference
│   manifest.json - The base assistant definition
│   TestAssistant.dll - A .NET 8 library we which contains the local functions in executable form
│
└───OpenAPI
        tool.localhost.json - a swagger file where the server url is "url": "tool://localhost" and Paths are names of static methods in any loaded assembly 
```        

### Explanation
AssistantUtility.Create acts based on the contents of a folder structure following a convention laid out in these notebooks. In this case, it copied the contents of instructions.md into the assistant manifest.json. It also parsed the tool.localhost swagger which follows an Open API schema into tool defintions for the assitant.

In [8]:
var assistantId = await AssistantUtility.Create("LocalCodeTool", config);
Console.WriteLine(assistantId)

asst_hd3Jta5sg7tShHQap1gSt3A1


## Run the Assistant

output.Dialog shows the conversation. 

In [9]:
var assistantRunOptions = new AssistantRunOptions() {
    AssistantName = "LocalCodeTool",
    Instructions = "Use both of your tools to answer these two questions: What is the date and what is eleven plus negative 42?",
    UseConversationEvaluator = false
};
var output = await AntRunnerLib.AssistantRunner.RunThread(assistantRunOptions, config);
Console.WriteLine(output.Dialog)


User: Use both of your tools to answer these two questions: What is the date and what is eleven plus negative 42?

Assistant: I called the tool named GetDateAndTime with these arguments:
```
{}
```
and got this result:
```
"Wednesday, September 11, 2024"
```

I called the tool named AddNumbers with these arguments:
```
{"a": 11, "b": -42}
```
and got this result:
```
-31
```

The current date is Wednesday, September 11, 2024, and eleven plus negative 42 is -31.



In [5]:
Console.WriteLine(output.LastMessage)

The current date is Wednesday, September 11, 2024, and eleven plus negative 42 is -31.
